lightcontourfinder() is a python code to calculate microlensed images of circles and the according magnification

Copyright 2020 Robert W. Schmidt rschmidthd at gmail.com

The code generalizes the [Witt (1993)](https://ui.adsabs.harvard.edu/abs/1993ApJ...403..530W/abstract) method to circular source tracks. countourfinder() calculates image tracks of circular sources by first calculating the image of a straight track and then calculating the images of circular tracks starting on the straight track. Critical curves are found by strong deviations of the images of nearby points on the source track.

Includes routines from https://github.com/rschmidthd/causticfinder-py and https://github.com/rschmidthd/lightcurvefinder . 

__Instructions__: you can run the notebook with two examples (switch example in cell [3]). It will produce an image, a caustic pattern (with track overlaid) and in one case a light curve (magnifications based on integrating the image tracks). You can change settings in the first few cells.

###### This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

###### This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

###### You should have received a copy of the GNU General Public License along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [ ]:
import matplotlib.pyplot as plt
import cmath as cm
import numpy as np
import time

%matplotlib inline

In [ ]:
# settings
#
#     Infrequently the code seems unable to exit loops. A smaller step size 
#     should fix this.

points_per_curve=200    # critical and caustic curve length = 2*nstars*points_per_curve
points_per_track=40001  # number of points on source track
track_length=10.0

In [ ]:
example=2

if example==1:

    # Check 1: Gould & Gaucherel (1997)

    stars_x,stars_y,masses = np.loadtxt('gould_gaucherel.txt',unpack=True)
    nstars=1
    sigma_c = 0
    gamma=0.6
    source_size=0.9
    start_point = 0.0 + 0.0j + source_size
    zeta0=start_point.real-track_length*1j
    zeta_track=np.linspace(zeta0,zeta0.conjugate(),points_per_track)

else:
    
    # Check 2: Beaulieu+ (2006)

    nstars=2
    sigma_c=0
    gamma=0
    stars_x=np.array([0.0,1.61])  
    stars_y=np.array([0.0,0.0])
    masses=np.array([1.0,0.000076])

    # track offset (in x) from the origin, circles start at the rightmost point (+1 radius)
    theta=2.756           # from East to North
    impact=0.359
    source_size=0.025
    track_offset=impact/np.sin(np.pi-theta) + source_size 

    # create linear source track
    zeta0=-10*np.cos(np.pi-theta)-10j*np.sin(np.pi-theta)
    zeta_track=np.linspace(zeta0+track_offset,zeta0*(-1)+track_offset,points_per_track)

    start_point=zeta_track[20090]                 # starting point, can pick any

straight_track=zeta_track[:]

In [ ]:
gamma = gamma / (1-sigma_c)   # reduced shear
normalize = cm.sqrt(abs(1-sigma_c))

# normalize star positions depending on sigma_c
stars=np.zeros(nstars)+np.zeros(nstars)*1j
if nstars>1:
    for i in range(nstars):
        stars[i]=stars_x[i]*normalize+stars_y[i]*normalize*1j
else:
    stars=stars_x*normalize+stars_y*normalize*1j

In [ ]:
# lens equation
def beta(z):
    return z + z.conjugate()*gamma + (masses/(stars-z).conjugate()).sum()
# poly_fraction1() and poly_fraction2() are
# auxiliary functions for the Newton-Raphson derivative
def poly_fraction1(z,phi,detj):
    sum1=(-2/(stars-z)).sum()
    sum2=(2*masses/(stars-z)**3).sum()
    sum3=(masses/(stars-z)**2).sum()     
    return sum1 + sum2/(gamma-cm.exp(phi*1j)*cm.sqrt(1-detj)+sum3)  # using Witt eq. 16
def poly_fraction2(z, phi, detj):
    sum1=(masses/(stars-z)**2).sum()
    sum2=(2*masses/(stars-z)**3).sum()    
    return (sum1 + gamma-cm.exp(phi*1j)*cm.sqrt(1-detj))/sum2       # using Witt eq. 16
def detj(z):
    fac=gamma+(masses/(((stars-z).conjugate())**2)).sum()
    return abs(1-fac*fac.conjugate())

def iso_magnification(detj):

    solns=np.zeros(2*nstars)+np.zeros(2*nstars)*1j
    critical_points=np.zeros(2*nstars*points_per_curve)+np.zeros(2*nstars*points_per_curve)*1j
    caustic_points=critical_points.copy()
 
    zp=0.5+0j
    
    # determine initial roots for phi=0
    for m in range(2*nstars):
        
        zp=zp*cm.exp(0.1j)  # allow Newton-Raphson some distance to converge
        norm=abs(zp)
    
        while (norm>1e-6):
            t=zp

            sum=0
            if (m>0):
                sum=(1/(t-solns[:m])).sum()

            zp = t - 1/(poly_fraction1(t,0,detj)-sum)

            norm=abs(t-zp)
        solns[m]=zp
        #print(m,solns)
        
    # follow curves for phi: 0 -> 2 pi
    for n in range(2*nstars):

        zp=solns[n]
        for m in range(points_per_curve):
            t=0
            phi=m*2*cm.pi/points_per_curve
            norm=abs(zp)
    
            while (norm>1e-6):
                t  = zp
                zp = t-poly_fraction2(t,phi,detj)
                norm = abs(t-zp)
            
            #z=beta(zp)
            critical_points[n*points_per_curve+m]=zp
            #caustic_points[n*points_per_curve+m]=z
    
    return critical_points

In [ ]:
# functions relating to the image track

def func(z,zeta):
    return (z+gamma*z.conjugate()+(masses/(stars-z).conjugate()).sum())-zeta
def dfunc(z):
    return (gamma+(masses/((stars-z).conjugate())**2).sum())


# -  Critical curves are found when the distance of images of consecutive points on the source track
#    diverge strongly (currently a factor 3) [still on this side of the cricial curve] -- or if
#    the iteration does not converge [already on the other side]

def follow_zeta_track(z,p,ct,verbose=False):

    flag=False
    dz_previous=10.0
    dz=1.0
    zp=z*cm.exp(0.1j)

    if ct>points_per_track-5:
        print ('near end of track ',ct)
    #print ('start with',ct+p,zeta_track[ct+p])
    
    while (dz<3*dz_previous):                     # criterion to stop at critical curve
        if (p==1) and (ct==len(zeta_track)-1):    # end of line
            flag=True
            break
        if ((p==-1) and ct==1):                   # came back to start
            flag=True
            break
            
        ct=ct+p                                   #  move backwards for p=-1
        if ct<0:                                  #  for circular source: if index negative,
                                                  #  go to other (periodic) side of track
            ct=ct+points_per_track

        #if ct>=points_per_track:                  # is this needed ever?
        #    ct=ct-points_per_track
            
        zeta=zeta_track[ct]
        
        tt=z
        dz_previous=dz
        norm=1.0
        while (norm>1e-6) and (norm < 100):    # convergence (at solution) or divergence (at critical curve)
            t=z
            f=func(t,zeta)
            df=dfunc(t)
            z=t-p*(f-df*f.conjugate())/detj(t)  # note f=zeta(z)-zeta_track[ct], cf Witt eq. 14
            norm=abs(t-z)
        dz=abs(z-tt)
        if verbose:
            print ('converged',z)
        
        if (dz<3*dz_previous):                  # criterion top stop at critical curve
            add_image_to_track(z,zeta,p,ct)
        #else:
            #print('critical line at',ct,p)
    ct=ct-p  # back to before the critical curve
    
    return flag,ct,zimg[-1]

def change_isomag_contour(z,p,ct,verbose=False):
    if verbose:
        print ('from',z,' ',end='')
    cr = iso_magnification(p*detj(z))
    sep=np.zeros(len(cr))
    zeta=beta(z)
    for x in range(len(cr)):
        sep[x]=abs(beta(cr[x])-zeta)
    zp=cr[np.argsort(sep)][0]
    if verbose:
        print ('jumped to ',zp,p,ct)
    return zp

def find_initial_primary_position(z, zeta):    # Witt eq. 11
    norm=1.0
    while (norm>1e-6):
        t=z
        z = zeta - gamma*t.conjugate()-(masses/(stars-t).conjugate()).sum()
        norm=abs(t-z)
    return z

def find_initial_secondary_position(zeta):    # Witt eq. 13
    sols=np.zeros(nstars)+np.zeros(nstars)*1j
    if nstars > 1:
        for x in range(nstars):
            sum=gamma*stars[x]
            for y in range(nstars):
                if y!=x:
                    sum=sum+(masses[y]/(stars[y]-stars[x]))
            sols[x]=stars[x]-masses[x]/(zeta.conjugate()-stars[x].conjugate()-sum)
    else:
        sum=gamma*stars
        sols=stars-masses/(zeta.conjugate()-stars.conjugate()-sum)
       
    return sols

def add_image_to_track(z,zeta,p,ct):
    zimg.append(z)
    pimg.append(p)
    zetasrc.append(zeta)
    
def find_indices(z,zeta_array,parity_array):    #  return list of indices of position on source track
                                                #  closest to z
    zeta=np.array(zeta_array)    
    result=[]
    dzeta=2*track_length/points_per_track
    for x in range(len(zeta)):
        dp=abs(zeta[x]-z)
        if dp<0.5*dzeta:
            result.append((x,parity_array[x]))
    return result

def measure_area(track):                        # very simple integrator -> sums up y_i*deltax_i
    track=np.array(track)
    x=track.real
    y=track.imag
    dx=x[1:]-x[:-1]
    ymean=0.5*(y[1:]+y[:-1])
    return (-1*(ymean*dx).sum())                # define clockwise = positive

In [ ]:
# lighttrackfinder main
#
# - The code first follows the primary track, which ends at either the track end or at a star.
#     If the primary track encounters a critical curve, the code will cross the critical curve,
#     and follow the track further (can happen several times).
# - Then the code follows all secondary image tracks, which all start near a star.
#     Secondary tracks are followed backwards to make sure that the primary track
#     also gets completed if it happened to end at a star.
#
# this procedure is a modified version of the one described in Witt 1993, ApJ 403, 530

def lighttrackfinder():
    #initial position primary track
    zeta=zeta_track[0]
    zp=find_initial_primary_position(zeta*cm.exp(0.01j), zeta)
   
    ct=0                           # ct is the source track index
    p=1
    add_image_to_track(zp,zeta,p,ct)

    flag,ct,z=follow_zeta_track(zp,p,ct)

    # if flag is False, the image track has hit a critical line
    while (not flag):
        # change isomag-line
        p=p*-1
        zp=change_isomag_contour(z,p,ct,verbose=True)

        # follow track
        flag,ct,z=follow_zeta_track(zp,p,ct)
    #print ('primary done')

    # initial positions secondary tracks
    zeta=zeta_track[-1]
    secondary_initial=find_initial_secondary_position(zeta)

    if nstars >1:
        for zp in secondary_initial:
            #print ('doing secondary')
            p=-1                       # negative parity images near the star positions
            ct=len(zeta_track)-1
            flag,ct,z=follow_zeta_track(zp,p,ct)
            while (not flag):
                p=p*-1
                zp=change_isomag_contour(z,p,ct,verbose=False)
            
                flag,ct,z=follow_zeta_track(zp,p,ct)
    else:
        #print ('doing secondary')
        zp=secondary_initial
        p=-1                       # negative parity images near the star positions
        ct=len(zeta_track)-1
        flag,ct,z=follow_zeta_track(zp,p,ct)
        while (not flag):
            p=p*-1
            zp=change_isomag_contour(z,p,ct,verbose=False)
            
            flag,ct,z=follow_zeta_track(zp,p,ct)
       
    #print ('track done')
    return flag


In [ ]:
def trace_circle(zp,p_start):
    #initial position primary track
    zeta=zeta_track[0]
    
    ct=0                           # ct is the source track index
    p=p_start
    add_image_to_track(zp,zeta,p,ct)

    flag,ct,z=follow_zeta_track(zp,p,ct,verbose=False)

    # if flag is False, the image track has hit a critical line
    while (not flag):
        # change isomag-line
        p=p*-1
        zp=change_isomag_contour(z,p,ct,verbose=False)

        # follow track
        flag,ct,z=follow_zeta_track(zp,p,ct,verbose=False)
    #print ('track done')
    return flag


def lightcontourfinder(pp):

    indices=find_indices(pp,zeta_straight,p_straight)
    #print(indices)
    
    for element in indices:        # working out the image tracks for each starting position corresponding
                                   # to the source position    start_point

        zimg.clear()
        pimg.clear()
        zetasrc.clear()
        
        index_start_point,p_start=element
        z_start=zimg_straight[index_start_point]
       
        status = trace_circle(z_start,p_start)
                   
        sub_track=zimg[:]          # new memory allocation to avoid saving identical pointers in img_stack
        img_stack.append(sub_track)

    return True

In [ ]:
# first run lighttrackfinder() to get to the start of the circle

zimg=[]
pimg=[]
zetasrc=[]
status = lighttrackfinder()

# save for later use, slice to make a copy
zimg_straight=zimg[:]
p_straight=pimg[:]
zeta_straight=zetasrc[:]


In [ ]:
# then run lightcontourfinder() to go round the circle
# all resulting image tracks are saved in the list object img_stack[]

points_per_track=20001
dphi=2*np.pi/(points_per_track)
phi=np.linspace(0,2*np.pi-dphi,points_per_track)
# circles start at the rightmost point
zeta_track=start_point+source_size*(np.cos(phi)-1)+source_size*np.sin(phi)*1j

img_stack=[]
%time status=lightcontourfinder(start_point)
print ('number of image tracks:',len(img_stack),'(good to check this number)')

In [ ]:
# plotting the image track (various colours), the stars (orange) and critical curves (green)
zimg=[]
number=0
colours=['b', 'r', 'c', 'm', 'y', 'k']

for el in img_stack:    
    zz=np.array(el)
    plt.plot(zz.real,zz.imag,'.',markersize=0.9,color=colours[number%6])
    number=number+1
    
critical = iso_magnification(0.0) 
for i in range(2*nstars):
    plt.plot(np.real(critical[i*points_per_curve:(i+1)*points_per_curve]),
             np.imag(critical[i*points_per_curve:(i+1)*points_per_curve]),color='green')

plt.scatter(stars_x,stars_y,color='orange')
#plt.ylim(-0.1,0.1)
#plt.xlim(1.5,1.7)

In [ ]:
# work out light curve based on integrating contours
# output is coordinate along track (Einstein radii)  ,  magnification  ,    time (seconds) for 100 measurements


if example==2:

    points_per_track=30001              # adjust as necessary
    
    area0=np.pi*source_size**2

    img_stack=[]
    solns_time=[]
    solns_mag=[]

    print ('light curve -- this takes some time')
    
    start = time.time()
    current=start

    for x in range(18000,22000,10):
    
        start_point=straight_track[x]
        dphi=2*np.pi/(points_per_track+1)
        phi=np.linspace(0,2*np.pi-dphi,points_per_track)
        zeta_track=start_point+source_size*(np.cos(phi)-1)+source_size*np.sin(phi)*1j

        img_stack=[]
        status = lightcontourfinder(start_point)
 
        area=0
        for el in img_stack:
            area=area+measure_area(el)         # counterclockwise = negative, clockwise = positive
        
        solns_time.append(x)
        solns_mag.append(area/area0)
        
        if x%1000==0:
            print (x,area/area0,end=' ')

            previous=current
            current=time.time()
            print (current-previous,'s')

    end = time.time()
    print ('total [h]',(end-start)/3600)

In [ ]:
if example==2:
    plt.plot(solns_time,np.array(solns_mag))

In [ ]:
# plotting the caustics and the track

critical = iso_magnification(0.0) 
caustic = []
for z in critical:
    caustic.append(beta(z))


for i in range(2*nstars):
    plt.plot(np.real(caustic[i*points_per_curve:(i+1)*points_per_curve]),
             np.imag(caustic[i*points_per_curve:(i+1)*points_per_curve]),color='green')

track1=(straight_track-1j*source_size-source_size) # top and bottom points of source
track2=(straight_track+1j*source_size-source_size)
plt.plot(track1.real,track1.imag,color='blue')
plt.plot(track2.real,track2.imag,color='blue')

#plt.xlim(0.8,1.2)
#plt.ylim(-0.2,0.2)